# Advanced Usage

This notebook will focus on some of the more complicated ways I have used TelFit in the past. The first thing will be generating a self-consistent fit to a full echelle spectrum. The most accurate way to do that is to simply loop over the orders, letting TelFit find the best parameters each time. However, if you want a self-consistent fit or want to go a bit faster, you can follow this approach:

1. Fit the several orders that are dominated by water lines
2. Fix the humidity to the average best-fit humidity 
3. Repeat steps 1-2 for any other molecules present (typically CO, CO2, N2O)
4. Generate a model spectrum for each order with the best-fit parameters.

I'll go back to the A-star spectrum to demonstrate this case

In [7]:
# Standard import bloack
from astropy.io import fits
import telfit
import numpy as np
import matplotlib.pyplot as plt

try:
    import seaborn as sns
    sns.set_context('notebook', font_scale=1.5)
    sns.set_style('whitegrid')
except ImportError:
    pass

%matplotlib inline

In [5]:
# Read in the data
hdulist = fits.open('../data/HIP_105891.fits')
orders = []
for hdu in hdulist[1:]:
    wl = hdu.data['wavelength']
    fl = hdu.data['flux']
    cont = hdu.data['continuum']
    error = hdu.data['error']
    data = telfit.DataStructures.xypoint(x=wl, y=fl, cont=cont, err=error)
    orders.append(data.copy())


In [37]:
# Initialize the fitter
fitter = telfit.TelluricFitter()
fitter.SetObservatory('McDonald')

# Get relevant info from the fits header. Note that I put these keys there...
header = fits.getheader('../data/HIP_105891.fits')
RH = header['HUMIDITY']
T_fahrenheit = header['AIRTEMP']
temperature = (T_fahrenheit - 32.0) * 5.0 / 9.0 + 273.15
P = header['BARPRESS'] * 33.77
angle = header['ZD']

# Tell TelFit about the values
fitter.AdjustValue(dict(temperature=temperature, pressure=P, angle=angle, resolution=45000))
fitter.FitVariable(dict(h2o=RH))
fitter.SetBounds(dict(h2o=(1, 99), co2=(100, 1000), ch4=(0.1, 10), 
                      n2o=(0.05, 5), co=(0.01, 1.0), resolution=(35000, 55000)))


For the most part, the setup is identical to the setup for fitting a single echelle order. What we will do next, however, fit each molecule separately for a few orders each. **This will take quite a while to run, so I don't suggest you do this during the tutorial!** 


In [38]:
# Define a class that we will use to hold the information for each molecule
class Parameter(object):
    def __init__(self, parname, order_list, guess):
        self.parname = str(parname)
        self.order_list = list(order_list)
        self.guess = float(guess)
        self.fitvals = [] # Will hold the best-fit parameters
        
    @property
    def propdict(self):
        return {self.parname: self.guess}
        
# Make parameters for each molecule
h2o = Parameter(parname='h2o', order_list=(2, 3, 33, 34, 35), guess=RH)
co2 = Parameter(parname='co2', order_list=(7, 8, 9, 10), guess=368.5)
ch4 = Parameter(parname='ch4', order_list=(12, 13, 14, 15), guess=1.8)
n2o = Parameter(parname='n2o', order_list=(30,), guess=0.32)
co = Parameter(parname='co', order_list=(38, 39, 40), guess=0.14)

# Adjust the value for each parameter to its initial guess
for par in (h2o, co2, ch4, n2o, co):
    fitter.AdjustValue(par.propdict)

In [ ]:
# Big, slow loop!
for par in (h2o, co2, ch4, co, n2o):
    for ordernum in par.order_list:
        order = orders[ordernum][100:-100]
        fitter.FitVariable(par.propdict)
        fitter.AdjustValue(dict(wavestart=order.x[0]-5, waveend=order.x[-1]+5))
        
        # Perform the fit
        source, model = fitter.Fit(data=order,                   # The data to fit
                                   resolution_fit_mode='SVD',   # This is best for IGRINS data. The other option is 'gauss'
                                   adjust_wave='data',          # Adjust the data wavelengths to match the model, rather than vice versa
                                   continuum_fit_order=4,       # Fit the continuum with a 4th-order polynomial
                                   wavelength_fit_order=3,      # Add a 3rd-order polynomial to the wavelengths
                                   air_wave=True,               # The wavelengths are in air (not vacuum)
                                   fit_source=True)             # Fit the source spectrum with a smoothing spline
        
        # Get the best-fit value
        best_fit = fitter.GetValue(par.parname)
        par.fitvals.append(best_fit)
        
    # Now that we have fit every order, fix the value to be the mean
    par.guess = np.mean(par.fitvals)
    fitter.AdjustValue(par.propdict)
    break